<a href="https://colab.research.google.com/github/yatharth230703/Aims_task3_r1_finalsub/blob/main/feature_extraction_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spotipy

In [1]:
import numpy as np
import pandas as pd
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [34]:
songs_path='/content/songs.json'
filtered_songdata_path='/content/filtered_songdata.json'

In [31]:
with open(songs_path) as f:
  songs = json.load(f)

In [32]:
print(len(songs))
songs

24999


['0pn7SynQPZBgNMlqPitrR6',
 '1JsLA3JWc4XE2StbJcRYG6',
 '23ssBTLuRpiE8hlCTEZCIk',
 '6BPJCWBiFHF7ItUQlyyP6x',
 '6GTVtjlDMEtnM2paZZ6VNe',
 '0qlCTm0U6Jcgx29HEYIXrg',
 '4a0sqvCBo7aRIS4GzBQpnp',
 '35QiTFf7gjbYmZ0Sj7dd3n',
 '2HEt32GOG4LhdkAcwkVDPD',
 '4kiwnJJ6ThoXzEean8UI1f',
 '4PCOSYkwl0aWdal9iZP1Oy',
 '1R7TwnwF1RuYNf5vXlpQUT',
 '2V65y3PX4DkRhy1djlxd9p',
 '6gTbAJOvivFXwkOEhSIOtS',
 '7gohMujUKNwS2ocu7I7yVW',
 '7n8lRa37iDNelsWw8JmoX4',
 '7vrJn5hDSXRmdXoR30KgF1',
 '3U70BInxXP4XBO7Bhyty7X',
 '6eCNz55YybdERmd840hKZa',
 '0IKc0xOUToF1PanrCgDUs1',
 '21hfiYaPDN8E14Trp4qTF3',
 '7jJ3EWr0umPsjC6CEupXRK',
 '4dHdqpGaO9km4TEw9nNQSR',
 '4DkjZPCTvs3W2VVyDi4MAt',
 '7pbg3ABlAZv2NiIdKbBBFm',
 '2wmrHfBFcCCYVdrlrUrJw3',
 '0OUQhRKgGSb0PPrLUuEQka',
 '49dFv4gH1SeY90FySDOwCE',
 '1vN8k0BhCIUrjsWCeAMpOW',
 '04aAxqtGp5pv12UXAg4pkq',
 '6Ec5LeRzkisa5KJtwLfOoW',
 '0FjBNvLsCCNu7vmCh8mZns',
 '7pG2i1QLokufVdQTbHsM0c',
 '0USw1cIvmZPOoU6pBGKWLR',
 '7A9EcBrwTG6bwswUy7beWW',
 '3CnK206lwL8j2ETlzBC9go',
 '2Tj9aHtf7g8ydYqsB6S9M5',
 

In [18]:
client_id = '499e8531441948d2b969c873772464d1'
client_secret = '5583ce79d5354c25a248d54ef7704ecb'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [19]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

def get_track_features(track_id):
    try:
        track = sp.track(track_id)
        audio_features = sp.audio_features(track_id)[0]

        features = {
            "Acousticness": audio_features.get('acousticness', 0),
            "Danceability": audio_features.get('danceability', 0),
            "Energy": audio_features.get('energy', 0),
            "Instrumentalness": audio_features.get('instrumentalness', 0),
            "Key": audio_features.get('key', 0),
            "Liveness": audio_features.get('liveness', 0),
            "Loudness": audio_features.get('loudness', 0),
            "Mode": audio_features.get('mode', 0),
            "Speechiness": audio_features.get('speechiness', 0),
            "Tempo": audio_features.get('tempo', 0),
            "Time Signature": audio_features.get('time_signature', 0),
            "Valence": audio_features.get('valence', 0),
            "Duration": track['duration_ms'] / 1000.0,
            "Explicit": int(track['explicit']),
            "Popularity": track['popularity']
        }


        return features
    except Exception as e:
        print(f"Error fetching features for {track_id}: {e}")
        return None
def extract_audio_features(audio_path, spotify_features, scaler=scaler, sr=22050, duration=30):
    try:
        y, sr = librosa.load(audio_path, sr=sr, duration=duration)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).T
        chroma = librosa.feature.chroma_stft(y=y, sr=sr).T
        spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr).T
        tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr).T
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr).T
        zcr = librosa.feature.zero_crossing_rate(y=y).T

        librosa_features = np.hstack([
            mfcc.mean(axis=0),
            chroma.mean(axis=0),
            spectral_contrast.mean(axis=0),
            tonnetz.mean(axis=0),
            rolloff.mean(axis=0),
            zcr.mean(axis=0)
        ])

        features = {
            'mode_confidence': np.nanmean(tonnetz),
            'key_confidence': np.nanmean(chroma),
            'tempo_confidence': np.nanmean(librosa.beat.beat_track(y=y, sr=sr)[0]),
            'melody': np.nanmean(librosa.feature.melspectrogram(y=y, sr=sr)),
            'harmony': np.nanmean(librosa.effects.hpss(y)[0]),
            'timbre': np.nanmean(librosa.feature.mfcc(y=y, sr=sr)),
            'beat_strength': np.nanmean(librosa.beat.beat_track(y=y, sr=sr)[0]),
            'chroma_features': np.nanmean(chroma),
            'spectral_contrast': np.nanmean(spectral_contrast)
        }

        artist_popularity = np.array(spotify_features.get('Artist Popularity', []))
        artist_followers = np.array(spotify_features.get('Artist Followers', []))

        if artist_popularity.size > 0 and artist_followers.size > 0:
            artist_rating = np.mean((artist_popularity + 10 * np.log1p(artist_followers)) / 100)
            features['Artist Rating'] = artist_rating
        else:
            features['Artist Rating'] = 0

        album_popularity = spotify_features.get('Popularity', 0)
        features['Album Rating'] = album_popularity / 100

        genres = spotify_features.get('Genres', [])
        if genres:
            genre_labels = label_encoder.transform(genres)
            genre_encoded = np.mean(genre_labels)
        else:
            genre_encoded = 0

        features['Genre'] = genre_encoded

        release_date_str = spotify_features.get('Release Date', '')
        if release_date_str:
            release_date = datetime.strptime(release_date_str, '%Y-%m-%d')
            days_since_release = (datetime.now() - release_date).days
            features['Days Since Release'] = days_since_release
        else:
            features['Days Since Release'] = 0

        spotify_features_dict = {
            'Acousticness': spotify_features.get('Acousticness', 0),
            'Danceability': spotify_features.get('Danceability', 0),
            'Energy': spotify_features.get('Energy', 0),
            'Instrumentalness': spotify_features.get('Instrumentalness', 0),
            'Key': spotify_features.get('Key', 0),
            'Liveness': spotify_features.get('Liveness', 0),
            'Loudness': spotify_features.get('Loudness', 0),
            'Mode': spotify_features.get('Mode', 0),
            'Speechiness': spotify_features.get('Speechiness', 0),
            'Tempo': spotify_features.get('Tempo', 0),
            'Time Signature': spotify_features.get('Time Signature', 0),
            'Valence': spotify_features.get('Valence', 0),
            'Duration': spotify_features.get('Duration', 0),
            'Explicit': int(spotify_features.get('Explicit', False)),
        }

        combined_features = np.hstack([
            librosa_features,
            list(features.values()),
            list(spotify_features_dict.values())
        ])

        return combined_features

    except Exception as e:
        print(features)
        print(spotify_features_dict)
        print(f"Error extracting features from {audio_path}: {e}")
        return None

In [ ]:
def prepare_features_dataframe(track_ids):
    features_list = []

    for track_id in track_ids:
        features = get_track_features(track_id)
        if features is not None:
            features_list.append(features)

    df = pd.DataFrame(features_list)
    return df

In [ ]:

df_features = prepare_features_dataframe(track_ids)
print(df_features)

In [21]:
def get_audio_analysis(track_id):
    return sp.audio_analysis(track_id)

track_id = '6NMtzpDQBTOfJwMzgMX0zl'
audio_analysis = get_audio_analysis(track_id)
print(audio_analysis['sections'])

[{'start': 0.0, 'duration': 10.16527, 'confidence': 1.0, 'loudness': -14.575, 'tempo': 148.356, 'tempo_confidence': 0.357, 'key': 3, 'key_confidence': 0.07, 'mode': 1, 'mode_confidence': 0.591, 'time_signature': 4, 'time_signature_confidence': 1.0}, {'start': 10.16527, 'duration': 13.79506, 'confidence': 0.894, 'loudness': -9.075, 'tempo': 147.176, 'tempo_confidence': 0.283, 'key': 10, 'key_confidence': 0.0, 'mode': 1, 'mode_confidence': 0.241, 'time_signature': 4, 'time_signature_confidence': 1.0}, {'start': 23.96034, 'duration': 27.56173, 'confidence': 0.467, 'loudness': -7.435, 'tempo': 148.101, 'tempo_confidence': 0.363, 'key': 5, 'key_confidence': 0.063, 'mode': 0, 'mode_confidence': 0.317, 'time_signature': 4, 'time_signature_confidence': 1.0}, {'start': 51.52207, 'duration': 86.56197, 'confidence': 0.579, 'loudness': -5.094, 'tempo': 148.13, 'tempo_confidence': 0.287, 'key': 0, 'key_confidence': 0.271, 'mode': 0, 'mode_confidence': 0.44, 'time_signature': 4, 'time_signature_conf

In [22]:
!pip install yt-dlp youtube-search-python
import yt_dlp as youtube_dl
from youtubesearchpython import VideosSearch
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver 

In [ ]:
def search_youtube(query, max_results=1):
    videos_search = VideosSearch(query, limit=max_results)
    results = videos_search.result()
    return results['result'][0]['link'] if results['result'] else None

def download_audio_from_youtube(youtube_url, output_dir, spotify_id):
    song_metadata = get_track_metadata(spotify_id)
    song_name = f"{song_metadata['name']}"
    sanitized_song_name = sanitize_filename(song_name)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    output_template = os.path.join(output_dir, f"{sanitized_song_name}.%(ext)s")

    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': output_template,
        'quiet': True
    }

    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

def fetch_and_store_songs(songs, output_dir='downloaded_songs'):
    for song_id in songs:
        song_metadata = get_track_metadata(song_id)
        song_name = f"{song_metadata['name']} {song_metadata['artists'][0]['name']}"
        youtube_url = search_youtube(song_name)
        if youtube_url:
            print(f"Downloading {song_name} from {youtube_url}")
            download_audio_from_youtube(youtube_url, output_dir, song_id)
        else:
            print(f"No YouTube video found for {song_name}")

def get_track_metadata(track_id):
    track = sp.track(track_id)
    return {
        'name': track['name'],
        'artists': [{'name': artist['name']} for artist in track['artists']]
    }
def sanitize_filename(name):
    return ''.join(c if c.isalnum() or c in (' ', '-', '_') else '_' for c in name).strip()


In [83]:
import yt_dlp as youtube_dl
from youtubesearchpython import VideosSearch
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datetime import datetime


In [ ]:
def get_genres_for_songs(song_ids):
    all_genres = set()

    for song_id in song_ids:
        try:
            track = sp.track(song_id)
            artist_id = track['artists'][0]['id']
            artist = sp.artist(artist_id)
            artist_genres = artist['genres']
            all_genres.update(artist_genres)
        except Exception as e:
            print(f"Error fetching genres for song {song_id}: {e}")

    all_genres_list = list(all_genres)
    return all_genres_list

all_possible_genres = get_genres_for_songs(songs)

In [94]:


label_encoder.fit(all_possible_genres)

LabelEncoder()

IndentationError: unexpected indent (<ipython-input-86-7e4a19b422c8>, line 2)

In [75]:
def extract_audio_features(audio_path, spotify_features, sr=22050, duration=30):
    try:
        y, sr = librosa.load(audio_path, sr=sr, duration=duration)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).T
        chroma = librosa.feature.chroma_stft(y=y, sr=sr).T
        spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr).T
        tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr).T
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr).T
        zcr = librosa.feature.zero_crossing_rate(y=y).T

        librosa_features = np.hstack([
            mfcc.mean(axis=0),
            chroma.mean(axis=0),
            spectral_contrast.mean(axis=0),
            tonnetz.mean(axis=0),
            rolloff.mean(axis=0),
            zcr.mean(axis=0)
        ])

        features = {
            'mode_confidence': np.nanmean(tonnetz),
            'key_confidence': np.nanmean(chroma),
            'tempo_confidence': np.nanmean(librosa.beat.beat_track(y=y, sr=sr)[0]),
            'melody': np.nanmean(librosa.feature.melspectrogram(y=y, sr=sr)),
            'harmony': np.nanmean(librosa.effects.hpss(y)[0]),
            'timbre': np.nanmean(librosa.feature.mfcc(y=y, sr=sr)),
            'beat_strength': np.nanmean(librosa.beat.beat_track(y=y, sr=sr)[0]),
            'chroma_features': np.nanmean(chroma),
            'spectral_contrast': np.nanmean(spectral_contrast)
        }

        artist_popularity = np.array(spotify_features.get('Artist Popularity', []))
        artist_followers = np.array(spotify_features.get('Artist Followers', []))

        if artist_popularity.size > 0 and artist_followers.size > 0:
            artist_rating = np.mean((artist_popularity + 10 * np.log1p(artist_followers)) / 100)
            features['Artist Rating'] = artist_rating
        else:
            features['Artist Rating'] = 0

        album_popularity = spotify_features.get('Popularity', 0)
        features['Album Rating'] = album_popularity / 100

        genres = spotify_features.get('Genres', [])
        if genres:
            genre_labels = label_encoder.transform(genres)
            genre_encoded = np.mean(genre_labels)
        else:
            genre_encoded = 0  # Default or unknown genre encoding

        features['Genre'] = genre_encoded

        release_date_str = spotify_features.get('Release Date', '')
        if release_date_str:
            release_date = datetime.strptime(release_date_str, '%Y-%m-%d')
            days_since_release = (datetime.now() - release_date).days
            features['Days Since Release'] = days_since_release
        else:
            features['Days Since Release'] = 0

        spotify_features_dict = {
            'Acousticness': spotify_features.get('Acousticness', 0),
            'Danceability': spotify_features.get('Danceability', 0),
            'Energy': spotify_features.get('Energy', 0),
            'Instrumentalness': spotify_features.get('Instrumentalness', 0),
            'Key': spotify_features.get('Key', 0),
            'Liveness': spotify_features.get('Liveness', 0),
            'Loudness': spotify_features.get('Loudness', 0),
            'Mode': spotify_features.get('Mode', 0),
            'Speechiness': spotify_features.get('Speechiness', 0),
            'Tempo': spotify_features.get('Tempo', 0),
            'Time Signature': spotify_features.get('Time Signature', 0),
            'Valence': spotify_features.get('Valence', 0),
            'Duration': spotify_features.get('Duration', 0),
            'Explicit': int(spotify_features.get('Explicit', False)),
        }

        combined_features = np.hstack([
            librosa_features,
            list(features.values()),
            list(spotify_features_dict.values())
        ])

        return combined_features

    except Exception as e:
        print(features)
        print(spotify_features_dict)
        print(f"Error extracting features from {audio_path}: {e}")
        return None

In [90]:


def get_spotify_features(track_id):
    track = sp.track(track_id)
    audio_features = sp.audio_features(track_id)[0]
    artist_ids = [artist['id'] for artist in track['artists']]
    artist_infos = [sp.artist(artist_id) for artist_id in artist_ids]

    features = {
        "Acousticness": audio_features.get('acousticness', 0),
        "Danceability": audio_features.get('danceability', 0),
        "Energy": audio_features.get('energy', 0),
        "Instrumentalness": audio_features.get('instrumentalness', 0),
        "Key": audio_features.get('key', 0),
        "Liveness": audio_features.get('liveness', 0),
        "Loudness": audio_features.get('loudness', 0),
        "Mode": audio_features.get('mode', 0),
        "Speechiness": audio_features.get('speechiness', 0),
        "Tempo": audio_features.get('tempo', 0),
        "Time Signature": audio_features.get('time_signature', 0),
        "Valence": audio_features.get('valence', 0),
        "Artists": [artist['name'] for artist in track['artists']],
        "Album Name": track['album']['name'],
        "Album Type": track['album']['album_type'],
        "Release Date": track['album']['release_date'],
        "Popularity": track['popularity'],
        "Duration": track['duration_ms'],
        "Explicit": track['explicit'],
        "Genres": sp.artist(track['artists'][0]['id']).get('genres', []),
        "Artist Popularity": [artist_info['popularity'] for artist_info in artist_infos],
        "Artist Followers": [artist_info['followers']['total'] for artist_info in artist_infos]
    }

    return features

In [ ]:
def fetch_and_store_songs(songs, output_dir='downloaded_songs', output_file='features.csv', batch_size=100):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if os.path.exists(output_file):
        df = pd.read_csv(output_file)
    else:
        df = pd.DataFrame()

    all_features = []
    processed_songs = set(df['song_id'].values) if not df.empty else set()

    for i, song_id in enumerate(songs):
        if song_id in processed_songs:
            print(f"Song {song_id} already processed. Skipping.")
            continue

        retries = 1000
        while retries > 0:
            try:
                song_metadata = get_track_metadata(song_id)
                song_name = f"{song_metadata['name']}".replace('/', '_')
                song_filename = os.path.join(output_dir, f"{song_name}.mp3")

                if not os.path.exists(song_filename):
                    print(f"Downloading {song_name}...")
                    youtube_url = search_youtube(song_name)
                    print(f"Downloading {song_name} from {youtube_url}")
                    download_audio_from_youtube(youtube_url, output_dir, song_id)


                print(f"Extracting features from {song_name}...")
                spotify_features = get_spotify_features(song_id)
                audio_features = extract_audio_features(song_filename, spotify_features)

                if audio_features is not None:
                    all_features.append([song_id] + list(audio_features))
                    processed_songs.add(song_id)
                else:
                    print(f"Failed to extract features from {song_name}.")

                break  # Exit retry loop if successful
            except Exception as e:
                retries -= 1
                print(f"Error processing {song_id}: {e}. Retries left: {retries}")
                time.sleep(2 ** (10 - retries))

        if len(all_features) >= batch_size:
            save_to_csv(all_features, df, output_file)
            all_features = []

    if all_features:
        save_to_csv(all_features, df, output_file)

    return df


In [ ]:
def save_to_csv(new_features, df, output_file):
    columns = [
        'song_id',
        # Librosa features
        'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13',
        'chroma1', 'chroma2', 'chroma3', 'chroma4', 'chroma5', 'chroma6', 'chroma7', 'chroma8', 'chroma9', 'chroma10', 'chroma11', 'chroma12',
        'spectral_contrast1', 'spectral_contrast2', 'spectral_contrast3', 'spectral_contrast4', 'spectral_contrast5', 'spectral_contrast6', 'spectral_contrast7',
        'tonnetz1', 'tonnetz2', 'tonnetz3', 'tonnetz4', 'tonnetz5', 'tonnetz6',
        'rolloff1',
        'zcr1',
        # Derived features
        'mode_confidence', 'key_confidence', 'tempo_confidence', 'melody', 'harmony', 'timbre', 'beat_strength', 'chroma_features', 'spectral_contrast',
        # Spotify features
        'Artist Rating', 'Album Rating', 'Genre', 'Days Since Release',
        'Acousticness', 'Danceability', 'Energy', 'Instrumentalness', 'Key', 'Liveness', 'Loudness', 'Mode', 'Speechiness', 'Tempo', 'Time Signature', 'Valence', 'Duration', 'Explicit'
    ]

    new_df = pd.DataFrame(new_features, columns=['song_id'] + columns[1:])
    combined_df = pd.concat([df, new_df], ignore_index=True)
    combined_df.to_csv(output_file, index=False)

In [91]:
df = fetch_and_store_songs(songs)
print(df)

In [93]:
print(df)

        mfcc1      mfcc2      mfcc3      mfcc4     mfcc5    mfcc6     mfcc7  \
0 -214.416016  77.215996  10.078969  22.095551  12.57605  2.61677  4.010253   

      mfcc8     mfcc9    mfcc10  ...   Key  Liveness  Loudness  Mode  \
0  7.353395  2.838591 -2.851131  ...  11.0     0.287    -3.544   0.0   

   Speechiness    Tempo  Time Signature  Valence  Duration  Explicit  
0        0.241  135.135             4.0    0.887   95555.0       0.0  

[1 rows x 67 columns]


In [62]:
def print_song_names(spotify_ids):
    """
    Print the names of the songs for a list of Spotify track IDs.

    Parameters:
    spotify_ids (list): A list of Spotify track IDs.
    """
    if not isinstance(spotify_ids, list):
        print("Error: spotify_ids must be a list.")
        return

    # Ensure ids are strings and not empty
    spotify_ids = [str(id) for id in spotify_ids if id]

    if not spotify_ids:
        print("No valid Spotify IDs provided.")
        return

    # Spotify API allows up to 50 track IDs at a time
    chunk_size = 50
    for i in range(0, len(spotify_ids), chunk_size):
        chunk = spotify_ids[i:i+chunk_size]
        try:
            results = sp.tracks(chunk)
            for track in results['tracks']:
                print(track['name'])
        except Exception as e:
            print(f"An error occurred: {e}")



In [65]:
print_song_names(songs[0:2])

Taquei na Muda
Cuatro Paredes


In [36]:
with open(filtered_songdata_path) as f:
  filtered_songdata = json.load(f)

In [41]:
print(filtered_songdata[3])

{'_id': {'$oid': '6439183f7b183b707047cc7a'}, 'SongID': '005lwxGU1tms6HGELIcUv9', 'SongData': {'title': 'I Kissed A Girl', 'artists': [{'id': '6jJ0s89eD6GaHleKKya26X', 'name': 'Katy Perry', 'genre': ['dance pop', 'pop']}], 'id': '005lwxGU1tms6HGELIcUv9', 'genre': ['dance pop', 'pop']}, '__v': 0}


In [71]:

track = sp.track('1xYpVRspgaLxdJLOTQkTjg')
artist = sp.artist(track['artists'][0]['id'])
album = sp.album(track['album']['id'])
print(track)

{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0Y5tJX1MQlPlqiwlOH1tJY'}, 'href': 'https://api.spotify.com/v1/artists/0Y5tJX1MQlPlqiwlOH1tJY', 'id': '0Y5tJX1MQlPlqiwlOH1tJY', 'name': 'Travis Scott', 'type': 'artist', 'uri': 'spotify:artist:0Y5tJX1MQlPlqiwlOH1tJY'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'BB', 'BZ', 'BT', 'BW', 'BF', 'CV', 'CW', 'D

In [ ]:
'''
        'Artist Popularity': artist['popularity'],
        'Artist Followers': artist['followers']['total'],
        'Popularity': track['popularity'],
        'Genres': artist['genres'],
        'Release Date': track['album']['release_date'],
        'Acousticness': track['acousticness'],
        'Danceability': track['danceability'],
        'Energy': track['energy'],
        'Instrumentalness': track['instrumentalness'],
        'Key': track['key'],
        'Liveness': track['liveness'],
        'Loudness': track['loudness'],
        'Mode': track['mode'],
        'Speechiness': track['speechiness'],
        'Tempo': track['tempo'],
        'Time Signature': track['time_signature'],
        'Valence': track['valence'],
        'Duration': track['duration_ms'],
        'Explicit': track['explicit']
'''